1) Finetune NER with Spacy on simple questions with base parameters.

base_config is taken for the following parameters (cpu, ner and accuracy as a target) from https://spacy.io/usage/training/;

the only change is vectors = "en_core_web_lg" is changed for vectors = "en_core_web_sm"

2) With the help of the saved model, prediction is made for all three datasets (test, train, dev)

In [ ]:
! pip install spacy-transformers

In [31]:
import spacy
from spacy.tokens import DocBin
import en_core_web_sm
import pandas as pd
from tqdm import tqdm
import numpy as np

### Prepare data for Spacy input

In [55]:
data_ner = pd.read_csv('val_ner_auto.csv')

In [3]:
training_data = []
for i in tqdm(range(len(data_ner))):
    init_question = data_ner.loc[i, 'question']
    subj = data_ner.loc[i, 'subject_text_all']
    cand = data_ner.loc[i, 'ner_auto']
    if pd.notna(cand) and pd.notna(subj):
        cand = " ".join(cand.split())
        ind = init_question.find(cand)
        training_data.append((init_question, [(ind, ind+len(cand), "PERSON")]))

100%|██████████| 2821/2821 [00:00<00:00, 39029.84it/s]


In [4]:
count = 0
nlp = spacy.blank("en")
# the DocBin will store the example documents
db = DocBin()
for text, annotations in training_data:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        count +=1 
        span = doc.char_span(start, end, label=label)
        if span is None:
            span = doc.char_span(start, end+1, label=label)
        if span is None:
            span = doc.char_span(start, end+2, label=label)
        if span is None:
            span = doc.char_span(start, end+3, label=label)
        if span is None:
            span = doc.char_span(start, end+4, label=label)
        if span is None:
            span = doc.char_span(start, end+5, label=label)
        if span is None:
            span = doc.char_span(start, end+6, label=label)
        if span is None:
            span = doc.char_span(start, end+7, label=label)
        if span is None:
            span = doc.char_span(start, end+8, label=label)
        if span is None:
            span = doc.char_span(start, end+9, label=label)
        if span is None:
            span = doc.char_span(start, end+10, label=label)
        if span is None:
            span = doc.char_span(start, end+11, label=label)
        if span is None:
            span = doc.char_span(start, end+12, label=label)
        if span is None:
            span = doc.char_span(start, end+13, label=label)
        if span is None:
            span = doc.char_span(start, end+14, label=label)
        if span is None:
            span = doc.char_span(start, end+15, label=label)
        if span is None:
            span = doc.char_span(start-1, end, label=label)
        if span is None:
            span = doc.char_span(start-2, end, label=label)
        if span is None:
            span = doc.char_span(start-3, end, label=label)
        if span is None:
            span = doc.char_span(start-2, end+1, label=label)
        if span is None:
            span = doc.char_span(start-4, end, label=label)
        if span is None:
            span = doc.char_span(start-5, end, label=label)
        if span is None:
            span = doc.char_span(start-6, end, label=label)
        if span is None:
            span = doc.char_span(start-7, end, label=label)
        if span is None:
            span = doc.char_span(start-8, end, label=label)
        if span is None:
            span = doc.char_span(start-9, end, label=label)
        if span is None:
            span = doc.char_span(start-10, end, label=label)
        ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./dev.spacy")

### Train model

In [5]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [6]:
! python3 -m spacy init fill-config data/base_config.cfg data/config.cfg

✔ Auto-filled config with all values
✔ Saved config
data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
nlp = en_core_web_sm.load()

In [9]:
! python3 -m spacy train data/config.cfg --output ./output --paths.train data/train.spacy --paths.dev data/dev.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-03 07:42:28,889] [INFO] Set up nlp object from config
[2022-10-03 07:42:28,898] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-10-03 07:42:28,901] [INFO] Created vocabulary
[2022-10-03 07:42:29,367] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2022-10-03 07:42:29,369] [INFO] Added vectors: en_core_web_sm
[2022-10-03 07:42:29,370] [INFO] Finished initializing nlp object
[2022-10-03 07:42:34,010] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  -----

### Prediction with finetuned NER

In [56]:
nlp = spacy.load("output/model-best")
for j in tqdm(range(len(data_ner))):
    cur_question = data_ner.loc[j, 'question']
    doc = nlp(cur_question)
    data_ner.loc[j, 'ner_spacy'] =','.join([ent.text for ent in doc.ents])
    if data_ner.loc[j, 'ner_spacy'] == '':
        data_ner.loc[j, 'ner_spacy'] = np.nan

100%|██████████| 2821/2821 [00:27<00:00, 103.75it/s]


In [57]:
#generate sentence with start-end tokens
for j in range(len(data_ner)):
    init_question = data_ner.loc[j, 'question']
    if pd.notna(data_ner.loc[j, 'ner_spacy']):
        cand = data_ner.loc[j, 'ner_spacy']
        ind = data_ner.loc[j, 'question'].find(cand)
        new_ques = init_question[:ind]+ '[START] '+ init_question[ind:ind+len(cand)] + ' [END]' + init_question[ind+len(cand):]
        data_ner.loc[j, 'question_ner_spacy'] = new_ques
    else:
        new_ques = '[START] '+ init_question +'[END]'
        data_ner.loc[j, 'question_ner_spacy'] = new_ques

In [58]:
data_ner.to_csv('val_ner_auto_spacy.csv')